# Managing State in AutoGen: A Tutorial on Saving and Loading Agent States

In this tutorial, we'll explore how to manage state in AutoGen applications, focusing on saving and loading states for agents and teams. This is particularly useful for web applications or scenarios where you need to persist conversation history.

In [ ]:
import os
import getpass
import json
from autogen import AssistantAgent, UserProxyAgent
from autogen.agentchat.contrib.agent_builder import AgentBuilder

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

## Basic Agent State Management

Let's start with a simple example of saving and loading an agent's state:

In [ ]:
# Create an assistant agent
assistant = AssistantAgent(
    name="poetry_assistant",
    system_message="You are a creative poetry assistant specializing in nature poems.",
    llm_config={
        "temperature": 0.7,
        "model": "gpt-4"
    }
)

# Create a user proxy agent
user = UserProxyAgent(
    name="user",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1
)

# Initialize a chat
chat_messages = assistant.initiate_chat(
    user,
    message="Write a haiku about mountains"
)

# Save the agent's state
agent_state = assistant.save_state()

# Print the state
print("Saved agent state:")
print(json.dumps(agent_state, indent=2))

## Loading Agent State

Now let's create a new agent and load the previous state:

In [ ]:
# Create a new assistant
new_assistant = AssistantAgent(
    name="poetry_assistant",
    system_message="You are a creative poetry assistant specializing in nature poems.",
    llm_config={
        "temperature": 0.7,
        "model": "gpt-4"
    }
)

# Load the previous state
new_assistant.load_state(agent_state)

# Test if the state was properly loaded
chat_messages = new_assistant.initiate_chat(
    user,
    message="What was the last poem you wrote?"
)

## Working with Team States

Let's explore how to manage state with multiple agents working as a team:

In [ ]:
# Create a team of agents
critic = AssistantAgent(
    name="poetry_critic",
    system_message="You are a poetry critic who provides constructive feedback.",
    llm_config={
        "temperature": 0.7,
        "model": "gpt-4"
    }
)

team_members = [assistant, critic]
team_config = {
    "agents": team_members,
    "max_rounds": 3
}

# Create some interaction between team members
initial_message = "Let's create and review a nature poem."
chat_messages = assistant.initiate_chat(
    critic,
    message=initial_message
)

# Save team state
team_state = {
    "assistant_state": assistant.save_state(),
    "critic_state": critic.save_state()
}

# Save to file
with open("team_state.json", "w") as f:
    json.dump(team_state, f)

## Loading Team State from File

In [ ]:
# Load team state from file
def load_team_state(filename):
    with open(filename, "r") as f:
        loaded_state = json.load(f)
    
    # Create new agents
    new_assistant = AssistantAgent(
        name="poetry_assistant",
        system_message="You are a creative poetry assistant specializing in nature poems.",
        llm_config={
            "temperature": 0.7,
            "model": "gpt-4"
        }
    )
    
    new_critic = AssistantAgent(
        name="poetry_critic",
        system_message="You are a poetry critic who provides constructive feedback.",
        llm_config={
            "temperature": 0.7,
            "model": "gpt-4"
        }
    )
    
    # Load states
    new_assistant.load_state(loaded_state["assistant_state"])
    new_critic.load_state(loaded_state["critic_state"])
    
    return new_assistant, new_critic

# Usage example
new_assistant, new_critic = load_team_state("team_state.json")

## Best Practices for State Management

In [ ]:
class StateManager:
    def __init__(self, storage_path):
        self.storage_path = storage_path
        
    def save_agent_state(self, agent, filename):
        state = agent.save_state()
        full_path = os.path.join(self.storage_path, filename)
        with open(full_path, "w") as f:
            json.dump(state, f)
            
    def load_agent_state(self, agent, filename):
        full_path = os.path.join(self.storage_path, filename)
        with open(full_path, "r") as f:
            state = json.load(f)
        agent.load_state(state)
        
    def cleanup_old_states(self, max_age_days=7):
        """Remove state files older than max_age_days"""
        current_time = time.time()
        for filename in os.listdir(self.storage_path):
            file_path = os.path.join(self.storage_path, filename)
            if os.path.getmtime(file_path) < current_time - (max_age_days * 86400):
                os.remove(file_path)

# Usage example
state_manager = StateManager("./agent_states")
state_manager.save_agent_state(assistant, "poetry_assistant_state.json")

This tutorial covered the essential aspects of state management in AutoGen, including saving and loading states for individual agents and teams, persisting states to disk, and implementing best practices for state management in production environments.